# Wu Chinese (wuu)

The Wu Chinese language is spoken in the region of China around Shanghai, but isn't heard much due to Chinese government regulations preventing broadcasts in languages other than Mandarin (cmn).

This means we need to get creative about how we source Wu Chinese samples.

## Phonemica

Phonemica is a site that allows people to record stories in their local dialect. We'll try to fetch samples from around Shanghai.

In [1]:
API_URL = 'http://phonemica.net/entry?id={0}'
START_ID = 2

In [2]:
import sys
from pyquery import PyQuery as pq
import parse
from collections import Counter
import urllib
import itertools
import datetime as dt

sys.path.append('../src')
import index

### Dialects available

Let's look at the first 100 dialects available.

In [3]:
def fix(s):
    return s.encode('latin-1').decode('utf-8')

In [4]:
START_ID = 2

def get_lang(doc):
    p = doc('label.managemuyu').closest('p')[0]
    text = fix(p.text_content())
    _, lang = text.split(':')
    return lang.strip()

def get_storyteller_url(i):
    doc = pq(API_URL.format(i))
    doc_s = pq.html(doc)
    url_end = parse.parse("{}window.location.href='{}';{}", doc_s).fixed[1]
    url = 'http://phonemica.net' + url_end
    return url

def iter_entries(start_id=START_ID):
    for i in itertools.count(start_id):
        yield get_storyteller_url(i)

In [5]:
dialects = Counter()

for url in itertools.islice(iter_entries(), 100):
    sys.stdout.write('.')
    sys.stdout.flush()
    try:
        doc = pq(url)
    except urllib.error.HTTPError:
        continue
    
    lang = get_lang(doc)
    dialects[lang] += 1

print()
dialects

....................................................................................................


Counter({'官话 普通话': 10, '官话 西南官话 湖广片': 6, '官话 西南官话 川黔片': 5, '吴语 太湖片 上海小片': 4, '粤语 广府片': 4, '吴语 太湖片 苏嘉湖小片': 4, '闽语 闽南片 泉漳小片': 3, '赣语 鹰弋片': 3, '官话 江淮官话 洪巢片': 3, '晋语 张呼片': 2, '赣语 大通片': 2, '吴语 宣州片 太高小片': 2, '官话 冀鲁官话 石济片': 2, '吴语 台州片': 2, '吴语 太湖片 毗陵小片': 2, '湘语 长益片 长株潭小片': 2, '客语 粤台片 梅惠小片': 2, '赣语 昌都片': 2, '官话 中原官话 郑开片': 2, '官话 中原官话西 秦晋片': 2, '湘语 邵祁片 东祁小片': 2, '晋语 邯新片': 1, '官话 江淮官话 通泰片': 1, '赣语 抚广片': 1, '吴语 太湖片 甬江小片': 1, '粤语 高阳片': 1, '官话 西南官话 桂柳片': 1, '吴语 金衢片': 1, '客语 海陆片': 1, '平话和土话 桂南片': 1, '客语 粤西片': 1, '吴语 上丽片 上山小片': 1, '闽语 莆仙片 莆田小片': 1, '湘语 湘中片 新化小片': 1, '赣语 宜浏片': 1, '晋语 大包片': 1, '官话 西南官话 西蜀片': 1, '官话 中原官话 徐淮片': 1, '官话 东北官话': 1, '晋语 上党片': 1, '吴语 太湖片 临绍小片': 1, '闽语 闽南片 潮汕小片': 1, '官话 兰银官话 银吴片': 1})

In [6]:
wu = {k: v for (k, v) in dialects.items()
      if k.startswith('吴语')}
wu

{'吴语 上丽片 上山小片': 1,
 '吴语 台州片': 2,
 '吴语 太湖片 上海小片': 4,
 '吴语 太湖片 临绍小片': 1,
 '吴语 太湖片 毗陵小片': 2,
 '吴语 太湖片 甬江小片': 1,
 '吴语 太湖片 苏嘉湖小片': 4,
 '吴语 宣州片 太高小片': 2,
 '吴语 金衢片': 1}

In [7]:
sum(wu.values())

18

So about 20% of the cases are Wu Chinese variants. Let's try fetching some of those.

### Fetching Wu audio

In [8]:
def get_media_url(storyteller_id, story_id):
    return 'http://phonemica.net/files/{0}/{1}.mp3'.format(storyteller_id, story_id)

In [9]:
def get_date(storyteller_id, story_id):
    url = 'http://phonemica.net/entry?mid={0}&story={1}'.format(storyteller_id, story_id)
    doc = pq(url)
    date_s = doc('small[style="float: right"]')[1].text_content()
    assert len(date_s.split()[0]) == 8
    year = int(date_s[:4])
    month = int(date_s[4:6])
    day = int(date_s[6:8])
    return dt.date(year, month, day)


def iter_wu_entries():
    for url in iter_entries():
        try:
            doc = pq(url)
        except urllib.error.HTTPError:
            continue

        lang = get_lang(doc)
        if lang.startswith('吴语'):
            yield url, doc


def index_wu_entries(max_samples=20):
    seen = index.scan()

    for i, (url, doc) in enumerate(iter_wu_entries()):
        if i >= max_samples:
            break
            
        a = doc('li')[0].find('.//a')
        storyteller_id, story_id = parse.parse('/entry?mid={}&story={}',
                                               a.attrib['href']).fixed
        title = fix(a.text_content())
        media_url = get_media_url(storyteller_id, story_id)
        date = get_date(storyteller_id, story_id)

        if url in seen or media_url in seen:
            print('{0}. {1} (skipped)'.format(i + 1, title), flush=True)
            continue

        print('{0}. {1}'.format(i + 1, title), flush=True)

        sample = {
            'language': 'wuu',
            'source_url': url,
            'source_name': 'Phonemica',
            'media_urls': [media_url],
            'title': title,
            'date': str(date)
        }
        staged = index.stage_audio(media_url, 'wuu')
        sample['checksum'] = staged.checksum

        if staged.checksum in seen:
            print('   SKIPPING: checksum already in index', flush=True)
            continue

        index.save(sample)

In [10]:
index_wu_entries()

1. 那种很怪的梦 (skipped)
2. 我爷爷奶奶的爱情故事 (skipped)
3. 金山农民画我们这里还有人画吗？ (skipped)
4. 浙江湖州的一個故事 (skipped)
5. 到上海白相必去個地方 (skipped)
6. 下一句：是不是在上网啊？ (skipped)
7. 南京高淳的一个故事 (skipped)
8. 海门的一个故事 (skipped)
9. 上海杨浦的一个故事 (skipped)
10. 绍兴的一个故事 (skipped)
11. 宁波的一个故事 (skipped)
12. 苏州的一个故事 (skipped)
13. 他一气，就亲自把那个人杀了 (skipped)
14. 无锡的一个故事 (skipped)
15. 上饶市的一个故事 (skipped)
16. 东阳的一个故事 (skipped)
17. 台州离家出走 (skipped)
18. 台州市的一个故事 (skipped)
19. 宜兴的一个故事 (skipped)
20. 苏州市的一个故事 (skipped)
